### This notebook shows examples of how to use the cyclops.query API on MIMIC-IV.

## Example 1. Get all patient encounters from 2015.

In [1]:
from sqlalchemy import select

from cyclops.query import mimic
from cyclops.query import utils as query_utils
from cyclops.query.interface import QueryInterface

patients = mimic.patients(years=[2015])
patients.run()
print(f"{len(patients.data)} rows extracted!")

2022-05-04 00:05:44,216 INFO cyclops.orm     - Database setup, ready to run queries!
2022-05-04 00:05:45,544 INFO cyclops.orm     - Query returned successfully!
2022-05-04 00:05:45,545 INFO cyclops.utils.profile - Finished executing function run_query in 1.322133 s


98084 rows extracted!


## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in ICD long title), in the year 2015.

In [2]:
patients = mimic.patients(years=[2015])
patients_schizophrenia = mimic.diagnoses(substring="schizophrenia", patients=patients)
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")
patients_schizophrenia.data["icd_title"].value_counts()

2022-05-04 00:05:46,236 INFO cyclops.orm     - Query returned successfully!
2022-05-04 00:05:46,237 INFO cyclops.utils.profile - Finished executing function run_query in 0.655030 s


5239 rows extracted!


Schizophrenia, unspecified                                                 3116
Unspecified schizophrenia, unspecified                                      954
Paranoid schizophrenia                                                      451
Unspecified schizophrenia, chronic                                          153
Paranoid type schizophrenia, unspecified                                    123
Catatonic schizophrenia                                                     103
Catatonic type schizophrenia, unspecified                                    65
Paranoid type schizophrenia, chronic with acute exacerbation                 52
Other schizophrenia                                                          43
Paranoid type schizophrenia, chronic                                         35
Other specified types of schizophrenia, chronic with acute exacerbation      26
Other specified types of schizophrenia, unspecified                          25
Catatonic type schizophrenia, chronic   

## Example 3. Advanced - uses regexp_match from sqlalchemy. Get all patient encounters with diagnoses (ICD long title contains `schizophrenia` and `chronic` ), in the year 2015.

In [3]:
patients = mimic.patients(years=[2015])
diagnoses = mimic.diagnoses(patients=patients)
subquery = (
    select(diagnoses.query)
    .where(diagnoses.query.c.icd_title.regexp_match("(?=.*schizophrenia)(?=.*chronic)"))
    .subquery()
)
patients_schizophrenia_chronic = diagnoses
patients_schizophrenia_chronic.query = subquery
patients_schizophrenia_chronic.run()
print(f"{len(patients_schizophrenia_chronic.data)} rows extracted!")

2022-05-04 00:05:47,353 INFO cyclops.orm     - Query returned successfully!
2022-05-04 00:05:47,354 INFO cyclops.utils.profile - Finished executing function run_query in 1.095409 s


310 rows extracted!


## Example 4. Advanced - uses cyclops.query.utils.starts_with helper function. Get all patient encounters with diagnoses (starts with `Paranoid` in ICD long title), in the year 2015.

In [4]:
patients = mimic.patients(years=[2015])
diagnoses = mimic.diagnoses(patients=patients)
subquery = (
    select(diagnoses.query)
    .where(query_utils.starts_with(diagnoses.query.c.icd_title, "Paranoid"))
    .subquery()
)
patients_schizophrenia = diagnoses
patients_schizophrenia.query = subquery
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")

2022-05-04 00:05:51,815 INFO cyclops.orm     - Query returned successfully!
2022-05-04 00:05:51,816 INFO cyclops.utils.profile - Finished executing function run_query in 4.443091 s


710 rows extracted!


## Example 5. Get all patient encounters with diagnoses (ICD code is F209).

In [5]:
patients = mimic.patients()
patients_f209 = mimic.diagnoses(diagnosis_codes=["F209"], patients=patients)
patients_f209.run()
print(f"{len(patients_f209.data)} rows extracted!")

2022-05-04 00:05:53,639 INFO cyclops.orm     - Query returned successfully!
2022-05-04 00:05:53,640 INFO cyclops.utils.profile - Finished executing function run_query in 1.810511 s


14818 rows extracted!


## Example 6. Get all patient encounters with diagnoses (`delirium` in ICD long title).

In [6]:
patients = mimic.patients()
patients_delirium = mimic.diagnoses(substring="delirium", patients=patients)
patients_delirium.run()
print(f"{len(patients_delirium.data)} rows extracted!")

2022-05-04 00:05:55,207 INFO cyclops.orm     - Query returned successfully!
2022-05-04 00:05:55,208 INFO cyclops.utils.profile - Finished executing function run_query in 1.559107 s


52482 rows extracted!


## Example 7. Get routine vital signs for patients from year 2015, limit to 100 rows.

In [7]:
patients = mimic.patients(years=[2015])
patients_vitals = mimic.events(category="Routine Vital Signs", patients=patients)
patients_vitals.run(limit=100)
print(f"{len(patients_vitals.data)} rows extracted!")

2022-05-04 00:07:22,241 INFO cyclops.orm     - Query returned successfully!
2022-05-04 00:07:22,245 INFO cyclops.utils.profile - Finished executing function run_query in 87.027072 s


100 rows extracted!


## Example 8. Get hemoglobin lab tests for patients from year 2009, limit to 100 rows.

In [8]:
patients = mimic.patients(years=[2009])
patients_hemo_labs = mimic.events(
    category="labs", patients=patients, names="hemoglobin"
)
patients_hemo_labs.run(limit=100)
print(f"{len(patients_hemo_labs.data)} rows extracted!")

2022-05-04 00:07:49,881 INFO cyclops.orm     - Query returned successfully!
2022-05-04 00:07:49,887 INFO cyclops.utils.profile - Finished executing function run_query in 27.629638 s


100 rows extracted!


## Example 9. Get all lab events for patients from year 2010, that match substring "sodium", limit to 100 rows.

In [9]:
patients = mimic.patients(years=[2009])
patients_sodium_labs = mimic.events(
    category="labs", patients=patients, substring="sodium"
)
patients_sodium_labs.run(limit=100)
print(f"{len(patients_sodium_labs.data)} rows extracted!")

2022-05-04 00:08:17,738 INFO cyclops.orm     - Query returned successfully!
2022-05-04 00:08:17,743 INFO cyclops.utils.profile - Finished executing function run_query in 27.846090 s


100 rows extracted!


## Example 10. Get respiratory events for patients from year 2009, limit to 100 rows.

In [10]:
patients = mimic.patients(years=[2009])
patients_respiratory = mimic.events(category="Respiratory", patients=patients)
patients_respiratory.run(limit=100)
print(f"{len(patients_respiratory.data)} rows extracted!")

2022-05-04 00:09:31,550 INFO cyclops.orm     - Query returned successfully!
2022-05-04 00:09:31,555 INFO cyclops.utils.profile - Finished executing function run_query in 73.802191 s


100 rows extracted!


## Example 11. Get heart rate measurements of patients with delirium diagnoses, limit to 1000 rows.

In [11]:
patients = mimic.patients()
patients_delirium = mimic.diagnoses(substring="delirium", patients=patients)
patients_delirium_heart_rate = mimic.events(
    names="Heart Rate", patients=patients_delirium
)
patients_delirium_heart_rate.run(limit=1000)
print(f"{len(patients_delirium_heart_rate.data)} rows extracted!")

2022-05-04 00:10:01,194 INFO cyclops.orm     - Query returned successfully!
2022-05-04 00:10:01,199 INFO cyclops.utils.profile - Finished executing function run_query in 29.610025 s


1000 rows extracted!


## Example 12. Get d_items dimension table, for quick reference.

In [12]:
event_lookup_table = mimic.get_lookup_table("event_labels")
lookup_data = event_lookup_table.run()
print(f"{len(lookup_data)} rows extracted!")

2022-05-04 00:10:01,222 INFO cyclops.orm     - Query returned successfully!
2022-05-04 00:10:01,223 INFO cyclops.utils.profile - Finished executing function run_query in 0.018595 s


3861 rows extracted!
